# HDBSCAN Clusterization
---

**HDBSCAN** merupakan pengembangan dari **DBSCAN**, dengan mengubahnya menjadi algoritma pengelompokan hierarkial,  ditambah dengan teknik untuk mengekstraksi pengelompokan yang didasarkan pada stabilitas suatu cluster - https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html

## 1. Load Dataset
---
Sebelumnya, saya sudah menyiapkan dataset berupa `5000` konten berita dari media online yang saya ambil per-tanggal 14 November 2019.

In [1]:
import json
from tqdm import tqdm

dataset_open = open("assets/dataset190521.jsonl", "r").readlines()

all_data = []
for i in tqdm(range(len(dataset_open))):
    data = dataset_open[i]
    loaded_data = json.loads(data)
    all_data.append(loaded_data)
    

100%|██████████| 5000/5000 [00:00<00:00, 26848.36it/s]


In [2]:
all_data[0]["content"]

'Korban ditemukan terbujur kaku sekitar pukul 13.00, Kamis (14/11) siang. Korban pertama kali ditemukan Gusti Nyoman Mandiasa, 61, warga Liligundi. Mandiasa saat itu hendak menuju kandang sapinya, yang terletak tak jauh dari lokasi kejadian. Saat itu Mandiasa mendapati sepeda kayuh milik korban, terparkir di Setra Desa Pakraman Beratan Samayaji. Semula Mandiasa tak menaruh curiga, karena di sekitar lokasi kejadian sering ada orang yang mencari pakan sapi. Saat berjalan, Mandiasa pun mendapati korban dalam kondisi terlentang.Saat diperiksa, ternyata tubuh korban dalam kondisi lemas dan tak ada denyut jantung. Mandiasa pun menghubungi kerabat korban dan menginformasikan bahwa korban telah meninggal dunia. Keluarga korban pun segera berdatangan ke lokasi kejadian. Selanjutnya pihak keluarga menghubungi petugas PMI dan membawa jenazah korban ke rumah duka di Lingkungan Penataran, Kelurahan Kendran. Salah seorang kerabat korban, Gusti Putu Kaler, 56, saat ditemui di rumah duka mengatakan, k

## 2. Preprocessing
---
Tahapan preprocessing merupakan yang dilakukan untuk membuat data mentah menjadi data yang berkualitas dengan menghilangkan beberapa kata yang tidak perlu, seperti stopwords, punctuationn (tanda baca) dan lain sebagainya sesuai kebutuhan. - https://en.wikipedia.org/wiki/Data_pre-processing

Dalam percobaan ini, saya menghilangkan `stopwords`, `punctuation` dan nomor, serta mengubah text menjadi `lowercase`

In [3]:
import re
import string

In [4]:
STOPWORD = open('assets/stopword.txt','r').read().split('\n')

In [5]:
def get_content_clean(text):
    text = text.replace("\n\n","")
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = re.sub('\d', ' ', text)
    text_ = []
    for t in text.split():
        if t not in STOPWORD:
            text_.append(t)

    return " ".join(text_)

In [6]:
data = []
all_content = []
for i in tqdm(range(len(all_data))):
    dt = all_data[i]
    
    data.append(dt)
    all_content.append(get_content_clean(dt["content_stopword"]))

100%|██████████| 5000/5000 [00:08<00:00, 562.02it/s]


Berikut adalah hasil dari tahapan *preprocessing*:

In [7]:
all_content[0]

'korban ditemukan terbujur kaku kamis siang korban kali ditemukan gusti nyoman mandiasa warga liligundi mandiasa kandang sapinya terletak lokasi kejadian mandiasa mendapati sepeda kayuh milik korban terparkir setra desa pakraman beratan samayaji mandiasa menaruh curiga lokasi kejadian orang mencari pakan sapi berjalan mandiasa mendapati korban kondisi terlentang diperiksa tubuh korban kondisi lemas denyut jantung mandiasa menghubungi kerabat korban menginformasikan korban meninggal dunia keluarga korban lokasi kejadian keluarga menghubungi petugas pmi membawa jenazah korban rumah duka lingkungan penataran kelurahan kendran salah kerabat korban gusti putu kaler ditemui rumah duka korban riwayat sakit an korban menglami stroke lumpuh terapi korban berhasil pulih kerja pensiun enam almarhum aktifitas tumben berangkat nyari rumput sakitnya kumat stroke riwayat jantung tanda tanda kekerasan kaler kasubbag humas polres buleleng iptu gede sumarjaya polisi pemeriksaan jenazah korban hasil peme

## 3. Build n-gram
---

Ngram merupakan sebuah metode yang diaplikasikan untuk pembangkitan kata atau karakter. 


Sehingga ketika ada sebuah kalimat : 

`Joko Widodo adalah presiden`

Dengan penerapan n-gram, dapat dideteksi bahwa kata Joko dan Widodo merupakan satu kesatuan, dan akan menjadi sebagai berikut:

`"Joko_Widodo" adalah presiden`

In [8]:
from gensim.models.phrases import Phrases, Phraser

In [9]:
sentences = []
for x in all_content:
    sentences.append(x.split())

In [10]:
phrases = Phrases(sentences, min_count=10, threshold=10)
bigram = Phraser(phrases)

trigram = Phrases(bigram[sentences],min_count=10, threshold=10)
trigram_ = Phraser(trigram)

In [11]:
contents_trigram = []
for x in sentences:
    tr = trigram_[bigram[x]]
    contents_trigram.append(" ".join(tr))

In [12]:
contents_trigram[0]

'korban ditemukan terbujur kaku kamis_siang korban kali ditemukan gusti nyoman mandiasa warga liligundi mandiasa kandang sapinya terletak lokasi_kejadian mandiasa mendapati sepeda kayuh milik korban terparkir setra desa pakraman beratan samayaji mandiasa menaruh curiga lokasi_kejadian orang mencari pakan sapi berjalan mandiasa mendapati korban kondisi terlentang diperiksa tubuh korban kondisi lemas denyut jantung mandiasa menghubungi kerabat korban menginformasikan korban_meninggal_dunia keluarga_korban lokasi_kejadian keluarga menghubungi petugas pmi membawa jenazah_korban rumah_duka lingkungan penataran kelurahan kendran salah kerabat korban gusti putu kaler ditemui rumah_duka korban riwayat sakit an korban menglami stroke lumpuh terapi korban berhasil pulih kerja pensiun enam almarhum aktifitas tumben berangkat nyari rumput sakitnya kumat stroke riwayat jantung tanda_tanda kekerasan kaler kasubbag humas_polres buleleng iptu gede sumarjaya polisi pemeriksaan jenazah_korban hasil_peme

## 4. TF-IDF Vectorize
---

Menggunakan TF-IDF untuk mengubah teks menjadi vector

In [13]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(
    min_df=5, 
    max_df=0.25
)

In [15]:
vector = vectorizer.fit_transform(contents_trigram)

In [16]:
vector.shape

(5000, 17002)

In [17]:
svd = TruncatedSVD(n_components=100, random_state=0)
small_vector = svd.fit_transform(vector)

small_vector.shape

(5000, 100)

## 5. HDBSCAN
---

Implementasi HDBSCAN untuk mendapatkan cluster dari kumpulan artikel yang sudah disiapkan sebelumnya.

In [18]:
import hdbscan
from collections import Counter

Parameter yang diset:
- `min_cluster_size` : jumlah artikel minimal yang dibutuhkan untuk membentuk suatu cluster
- `min_samples` : jumlah artikel yang digunakan sebagai perbandingan dengan artikel yang lain

In [19]:
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=5,
    min_samples=2
)

In [20]:
clusterer.fit(small_vector)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_method='eom',
        core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='euclidean', min_cluster_size=5, min_samples=2, p=None,
        prediction_data=False)

In [21]:
hdblabel = clusterer.labels_
n_clusters_ = len(set(hdblabel)) - (1 if -1 in hdblabel else 0)
print("Total cluster :",n_clusters_)

Total cluster : 218


Dari 5000 artikel yang sudah disiapkan, ternyata terbentuk 218 cluster

In [22]:
hdb_label = Counter(hdblabel)
hdb_label.most_common(10)

[(-1, 2675),
 (216, 108),
 (115, 67),
 (189, 63),
 (96, 62),
 (48, 46),
 (79, 43),
 (129, 31),
 (88, 31),
 (130, 29)]

In [23]:
all_cluster = {}
for i,d in enumerate(data):
    cluster_label = clusterer.labels_[i]
    if cluster_label in all_cluster and cluster_label not in [-1]:
        all_cluster[cluster_label].append(d["title"])
    elif cluster_label not in [-1]:
        all_cluster[cluster_label] = [d["title"]]

In [24]:
all_cluster_list = [v for k,v in all_cluster.items()]

### RESULT
Berikut ini adalah beberapa *sample* hasil dari proses *clustering* menggunakan **HDBSCAN**:

In [25]:
for cluster in all_cluster_list[:5]:
    print("-"*50)
    print()
    for title in cluster[:10]:
        print(title)
    print()

--------------------------------------------------

Duh Gusti, Lagi Ngarit Kena Serangan Jantung, Pensiunan Tewas
Pasca Bom Medan, Begini Kondisi Para Korbannya Saat ini | News | Arah.Com
Tersengat Listrik, Pekerja Las Tewas Kejang-Kejang | Radar Banjarmasin
Urat Otot Putus, Korban Bom Bunuh Diri Di Medan Jalani Operasi
Lebih Empat Jam Bencana, Arif Ditemukan Meninggal Dunia
Menegangkan, Detik-detik Basarnas Jabar Evakuasi Warga Jatinangor dari Dalam Sumur

--------------------------------------------------

BMKG: Gempa Buleleng Gempa Bumi Dangkal Akibat Aktivitas Sesar
Hoaks Kabar Tsunami Pasca Gempa, Warga Buleleng Berhamburan Keluar Rumah
Gempa Buleleng, Warga 4 Desa Sempat Ngungsi
Hoaks Kabar Tsunami Pasca Gempa, Warga Buleleng Berhamburan Keluar Rumah
Episentrum Gempa Bumi Dangkal secara Beruntun Dekat Perkampungan Buat Warga Panik
Dipicu Isu Tsunami, Warga Seririt Panik dan Mengungsi | wartabalionline.com | Jendela Pulau Dewata
Tidak Berpotensi Tsunami, BPBD Bali Minta Warga Seri

## 6. Visualization
---

Hasil cluster tadi dapat kita visualisasikan dalam tampilan dua dimensi, dibawah ini saya menggunakan library **Bokeh.JS** dan **TSNE** untuk mengubah vector yang sudah dibuat kedalam bentuk 2D.

In [26]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, verbose=0, random_state=0, n_iter=500)
tsne_tfidf = tsne_model.fit_transform(small_vector)

print(tsne_tfidf.shape)

(5000, 2)


In [27]:
import numpy as np
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.palettes import Category20
import seaborn as sns
output_notebook()

Loading BokehJS ...

In [28]:
X, Y = [], []
for x,y in tsne_tfidf:
    X.append(x)
    Y.append(y)

In [29]:
outliers = [hdb_label.most_common(1)[0][0]]

In [30]:
color_palette = sns.color_palette('Paired', clusterer.labels_.max()+1).as_hex()
cluster_colors = [color_palette[x] if x not in outliers else '#fff' for x in clusterer.labels_]

In [31]:
desc = []
id_cluster = []
for i,d in enumerate(data):
    desc.append(d["title"])
    id_cluster.append(clusterer.labels_[i])

In [32]:
source = ColumnDataSource(data=dict(
    x=X,
    y=Y,
    title=desc,
    color=cluster_colors,
    id_c=id_cluster
))

TOOLS = "hover,wheel_zoom,zoom_in,zoom_out"
TOOLTIPS = [
    ("Cluster", '@id_c'),
    ("Title", "@title")
]

p = figure(plot_width=900, plot_height=700, tools = TOOLS, tooltips=TOOLTIPS)

p.circle('x', 'y', size=7, source=source, alpha=0.5, color='color')

GlyphRenderer(id='1034', ...)

In [33]:
show(p)